In [1]:
import PyQt5
from PyQt5 import Qt
Qt.QT_VERSION_STR
FORCE_ENABLE_INLINE = False

In [2]:
import os
import sys
import gin
import numpy as np
import pandas as pd
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import matplotlib
matplotlib.use('Qt5Agg')

%matplotlib qt
#%matplotlib inline

import matplotlib.pyplot as plt
#import matplotlib.pyplot as plt
    
%load_ext autoreload
%autoreload 2

In [3]:
params = {
                    "sep": '\s+',
                   # "nrows": 10000,
                    "encoding": 'utf-8',
                   "names":  ['event',  'ntrack', 'det','station', 'side', 'z', 'y', 'x', 'time','dX', 'dY', 'dZ', 'dTime']
                 }
data = pd.read_csv("../data_cbm/hits10_withStations.txt", **params)
data

,event,ntrack,det,station,side,z,y,x,time,dX,dY,dZ,dTime
0,0,0,0,0,0,189.10,-6.511201,37.892959,-1.500000,0.000000,0.192039,0.192039,4.000000
1,0,0,0,0,0,189.10,1.125580,51.584594,-0.500000,0.000000,0.257713,0.257713,4.000000
2,0,0,0,0,0,189.10,-9.962932,52.476272,0.500000,0.000000,0.266788,0.266788,4.000000
3,0,0,0,0,0,189.10,-11.953212,39.431012,8.500000,0.000000,0.183792,0.183792,3.572228
4,0,0,0,0,0,189.10,-0.469619,35.874186,1.500000,0.000000,0.179198,0.179198,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2578423,999,1,2,0,0,867.19,201.003000,60.939840,30.982184,0.288675,0.135316,0.720000,0.080000
2578424,999,1,2,0,0,853.57,-59.802354,42.676923,29.158208,0.288675,0.135316,0.720000,0.080000
2578425,999,1,2,0,0,862.65,-66.919929,51.057060,553.672254,0.288675,0.135316,0.720000,0.080000
2578426,999,1,2,0,0,853.57,-60.823959,107.611993,29.631995,0.288675,0.135316,0.720000,0.080000


In [12]:
def visualize_simple_3d(data, axes=['x','y','z'], 
                        titleAx=['X', 'Y', 'Z'], 
                        title=None, 
                        scatter_params=None,
                        col_arr=None,
                        custom_draw=None):
    fig = plt.figure(figsize=(6,4))
    ax = fig.add_subplot(111, projection='3d')
    if custom_draw is not None:
        custom_draw(ax)
    else:
        if not scatter_params:
            scatter_params = {}
        if col_arr is None:        
            ax.scatter(data[axes[2]], data[axes[1]], data[axes[0]], **scatter_params)
        else:
            ax.scatter(data[axes[2]], data[axes[1]], data[axes[0]], c=col_arr, **scatter_params)
    ax.set_xlabel(titleAx[2])
    ax.set_ylabel(titleAx[1])
    ax.set_zlabel(titleAx[0])
    if title:
        plt.title(title)
    plt.tight_layout()
    plt.show()

In [5]:
print(data.station.unique(),
      data.det.unique(),
      data.side.unique())

[ 0  1  2  3  4  5  6  7  8  9 10 11] [0 1 2] [0 1]


In [7]:
# max_stat = data.station.max()
# max_side = data.side.max()

# data['colr'] = (data.station.values * (1./max_stat) ) 
# data['colg'] = 0 * (data.side.values * (1./max_side) )
# data['colb'] = 0 * 0.5 * data.colr.values

true_events = data[data.ntrack >= 2]
false_events = data[data.ntrack < 2]

one_true_event = true_events[true_events.event == true_events.event.unique()[0]]
one_false_event = false_events[false_events.event == false_events.event.unique()[0]]

In [8]:
visualize_simple_3d(one_true_event, title='True event', 
                    scatter_params={'s':10},
                    col_arr=one_true_event[['colr', 'colg', 'colb']].values
                   )
#visualize_simple_3d(one_false_event, title='False event', scatter_params={'c':'tab:red', 's':5})

KeyError: "None of [Index(['colr', 'colg', 'colb'], dtype='object')] are in the [columns]"

In [15]:
def custom_draw(ax, event=None):
    #event = event[(event.station == 11) | (event.station == 10)| (event.station == 0)]
    for station_id, by_station in event.groupby('station'):
        for side_id, by_side in by_station.groupby('side'):
            for det_id, by_det in by_side.groupby('det'):
                lab = "det #%d station #%d side #%d" % (det_id, station_id, side_id)
                ax.scatter(by_det.z, by_det.y, by_det.x, label=lab, s=35)
    #ax.legend(bbox_to_anchor=(1.1, 1.05))

from functools import partial


visualize_simple_3d(None, title='True event det0', 
                    custom_draw=partial(custom_draw, event=one_true_event[one_true_event.det == 0]))
visualize_simple_3d(None, title='True event det1',
                    custom_draw=partial(custom_draw, event=one_true_event[one_true_event.det == 1]))
visualize_simple_3d(None, title='True event det2',
                    custom_draw=partial(custom_draw, event=one_true_event[one_true_event.det == 2]))



In [16]:
def custom_draw(ax, event=None):
    #event = event[(event.station == 11) | (event.station == 10)| (event.station == 0)]
    for station_id, by_station in event.groupby('station'):
        for side_id, by_side in by_station.groupby('side'):
            for det_id, by_det in by_side.groupby('det'):
                lab = "det #%d station #%d side #%d" % (det_id, station_id, side_id)
                ax.scatter(by_det.z, by_det.y, by_det.x, label=lab, s=35)
    #ax.legend(bbox_to_anchor=(1.1, 1.05))

from functools import partial


visualize_simple_3d(None, title='False event det0', 
                    custom_draw=partial(custom_draw, event=one_false_event[one_false_event.det == 0]))
visualize_simple_3d(None, title='False event det1',
                    custom_draw=partial(custom_draw, event=one_false_event[one_false_event.det == 1]))
visualize_simple_3d(None, title='False event det2',
                    custom_draw=partial(custom_draw, event=one_false_event[one_false_event.det == 2]))

